In [468]:
# @Author: Kyunghun Lee 
# Updated: Fri Jan  5 09:50:58 EST 2024 by KL: all pracite-related data/result removed.
# Updated: Thu Dec 21 08:25:22 EST 2023 by KL: Bug fixed. feature added ('Go % Incorrect','Go % Omission', 'Stop % Incorrect').
# Updated: Tue Nov  7 15:53:36 EST 2023 by KL
# Created: Fri Nov  3 23:18:13 EDT 2023

import pandas as pd
import numpy as np

# Load the Excel file into a DataFrame
# df = pd.read_csv('input/24546_10.22.23_AtHome_S2.csv')
# df = pd.read_csv('input/24546_10.16.23_InPerson_S1.csv')
# df = pd.read_csv('input/taskSummary.csv')
# inFile = 'input/24626_9.6.23_InPerson_S1.csv'
df = pd.read_csv('input/24626_9.6.23_InPerson_S1.csv')
display(df)

Section Start Time Section End Time  Duration (ms)  \
0               05:33.7          29:44.8       69851108   
1               29:44.8          30:04.1          19341   
2               30:04.1          30:09.7           5566   
3               30:09.7          30:12.5           2767   
4               30:12.5          30:19.9           7468   
...                 ...              ...            ...   
1589            51:17.6          51:24.8           7277   
1590            51:24.8          51:27.8           3016   
1591            51:27.8          51:30.9           3034   
1592            51:30.9          51:38.2           7300   
1593            51:38.2          51:42.3           4100   

                                  Event                   Section  \
0     Subject Information Input Section  Getting user information   
1                   Slider View - tense                    Slider   
2                   Slider View - angry                    Slider   
3                 Slider View - annoyed                    Slider   
4                   Slider View - happy                    Slider   
...                                 ...                       ...   
1589                Slider View - tense                    Slider   
1590                Slider View - angry                    Slider   
1591              Slider View - annoyed                    Slider   
1592                Slider View - happy                    Slider   
1593                Slider View - tired                    Slider   

     User Response Correct Key Correctness  StimulusRT  InhibitRT  ...  \
0       sdan:24626         NaN         NaN         NaN        NaN  ...   
1               13         NaN         NaN         NaN        NaN  ...   
2                0         NaN         NaN         NaN        NaN  ...   
3                2         NaN         NaN         NaN        NaN  ...   
4               67         NaN         NaN         NaN        NaN  ...   
...            ...         ...         ...         ...        ...  ...   
1589            40         NaN         NaN         NaN        NaN  ...   
1590            50         NaN         NaN         NaN        NaN  ...   
1591            60         NaN         NaN         NaN        NaN  ...   
1592            30         NaN         NaN         NaN        NaN  ...   
1593            43         NaN         NaN         NaN        NaN  ...   

      PerRunGoRTAve  TrialNumber  PerRunCorrect PerRunIncorrect  InhibitACC  \
0           0.00000            0              0               0           0   
1           0.00000            0              0               0           0   
2           0.00000            0              0               0           0   
3           0.00000            0              0               0           0   
4           0.00000            0              0               0           0   
...             ...          ...            ...             ...         ...   
1589      951.50291           88             17               5           1   
1590      951.50291           88             17               5           1   
1591      951.50291           88             17               5           1   
1592      951.50291           88             17               5           1   
1593      951.50291           88             17               5           1   

      PerRunAcc  StimDurSum  StimDurAvg  PerRunStimDurAvg  DurationChange  
0      0.000000           0           0          0.000000             NaN  
1      0.000000           0           0          0.000000             NaN  
2      0.000000           0           0          0.000000             NaN  
3      0.000000           0           0          0.000000             NaN  
4      0.000000           0           0          0.000000             NaN  
...         ...         ...         ...               ...             ...  
1589   0.772727       12450           0        565.909091             NaN  
1590   0.772727      

In [469]:
# Remove rows where 'Event' column contains 'Practice'
df = df[~df['Section'].str.contains('Practice')]

In [470]:
print(df.columns)

Index(['Section Start Time', 'Section End Time', 'Duration (ms)', 'Event',
       'Section', 'User Response', 'Correct Key', 'Correctness', 'StimulusRT',
       'InhibitRT', 'SDAN', 'App Version', 'App Build', 'Inhibition',
       'StimulusDuration', 'InhibitDuration', 'AdjustmentAmount',
       'RunningCorrect', 'RunningIncorrect', 'RunningAccuracy', 'PerRunGoRT',
       'PerRunGoRTAve', 'TrialNumber', 'PerRunCorrect', 'PerRunIncorrect',
       'InhibitACC', 'PerRunAcc', 'StimDurSum', 'StimDurAvg',
       'PerRunStimDurAvg', 'DurationChange'],
      dtype='object')


In [471]:
# Extract VAS responses
dict_VAS = {}

# Filter rows where 'Event' contains 'Slider View'
filtered_df = df[df['Event'].str.contains('Slider View')]

# Extract the name and store the result in a dictionary
for _, row in filtered_df.iterrows():
    name = (row['Event'].split('Slider View - ')[1]).capitalize()
    if 'Pre_'+name not in dict_VAS:
        dict_VAS['Pre_'+name] = row['User Response']
    elif 'Mid_'+name not in dict_VAS:
        dict_VAS['Mid_'+name] = row['User Response']
    elif 'Post_'+name not in dict_VAS:
        dict_VAS['Post_'+name] = row['User Response']
    else:
        print("Error: More than 3 VAS scores found!")

# Initialize temporary dictionaries to store the sum and counts
emotion_sums = {}
emotion_counts = {}

# Process each item in the dictionary to accumulate sums and counts
for key, value in dict_VAS.items():
    # Isolate the emotion part of the key
    emotion = key.split('_')[1]
    
    # Convert the string value to float for computation
    value = float(value)
    
    # Accumulate sums and counts
    emotion_sums[emotion] = emotion_sums.get(emotion, 0) + value
    emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1

# Calculate the averages and update the original dict_VAS with new keys for averages
for emotion in emotion_sums:
    average_key = f'Avr_{emotion}'
    # Calculate the average and update dict_VAS
    dict_VAS[average_key] = emotion_sums[emotion] / emotion_counts[emotion]

display(dict_VAS)

{'Pre_Tense': '13',
 'Pre_Angry': '0',
 'Pre_Annoyed': '2',
 'Pre_Happy': '67',
 'Pre_Tired': '16',
 'Mid_Tense': '54',
 'Mid_Angry': '30',
 'Mid_Annoyed': '50',
 'Mid_Happy': '51',
 'Mid_Tired': '36',
 'Post_Tense': '40',
 'Post_Angry': '50',
 'Post_Annoyed': '60',
 'Post_Happy': '30',
 'Post_Tired': '43',
 'Avr_Tense': 35.666666666666664,
 'Avr_Angry': 26.666666666666668,
 'Avr_Annoyed': 37.333333333333336,
 'Avr_Happy': 49.333333333333336,
 'Avr_Tired': 31.666666666666668}

In [472]:
# Output header
headers = ['SDAN', 'Input File','Home vs InPerson','S1 vs S2',
           # 'Average Practice Go RT', 'Go Practice RT StDev',
       'Average Run Go RT', 'Stop % Accuracy', 'Stop % Incorrect', 'Go % Accuracy', 'Go % Incorrect','Go % Omission','Median Go RT',
       'Average SSD', 'SSRT', 'Post Go Error Go % Accuracy',
       'Post Go Error Go RT', 'Post Go Error Efficiency',
       'Post Any Error Go % Accuracy', 'Post Any Error Go RT',
       'Post Any Error Efficiency', 'Frustration Rating Block 1',
       'Frustration Rating Block 2', 'Frustration Rating Block 3',
       'Frustration Rating Block 4', 'Frustration Rating Block 5',
       'Average Frustration Rating', 'Happiness Rating Block 1',
       'Happiness Rating Block 2', 'Happiness Rating Block 3',
       'Happiness Rating Block 4', 'Happiness Rating Block 5',
       'Average Happiness Rating', 'Include/Exclude']

In [473]:
# Determine which RT to use based on conditions
conditions = [
    (df['Inhibition'] == 'Inhibit'),
    (df['Inhibition'] == 'Normal')
]

# Replace values in 'RT' column with values divided by 1000, only for non-NaN cells
df['StimulusRT'] = df['StimulusRT'].apply(lambda x: x if not pd.isna(x) else x)
df['InhibitRT'] = df['InhibitRT'].apply(lambda x: x if not pd.isna(x) else x)


choices = [df['InhibitRT'], df['StimulusRT']]
df['RT_to_use'] = np.select(conditions, choices, default=pd.NA)

# Convert to numeric, just in case they are not
df['RT_to_use'] = pd.to_numeric(df['RT_to_use'], errors='coerce')

# Remove rows where 'RT_to_use' is empty
df = df.dropna(subset=['RT_to_use'])
display(len(df))
# display(df['TrialNumber'])

550

In [474]:
import pandas as pd
import numpy as np

# Initialize the result dictionary
result = {}

In [475]:
# Remove rows where 'Correctness' column is not 'Correct' or 'Incorrect'
df = df[df['Correctness'].isin(['Correct', 'Incorrect'])]

# Shift the 'Correctness' column down by one to compare with the previous row
df['Post Error'] = df['Correctness'].shift(1) == 'Incorrect'
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # display(df)

In [476]:
# List of substrings (blocks) you are interested in
# blocks = ['Practice 1', 'Practice 2', 'Block 1', 'Block 2', 'Block 3', 'Block 4', 'Block 5']
blocks = ['Block 1', 'Block 2', 'Block 3', 'Block 4', 'Block 5']

# Define a function that takes a section string and returns the block
def find_block(section):
    for block in blocks:
        if block in section:
            return block
    return None  # Return None or some default value if no block is found

# Apply the function and assign using .loc
df.loc[:, 'Block'] = df['Section'].apply(find_block)
df['Block'] = df['Section'].apply(find_block)


# # List of substrings you are interested in
# substrings = ['Practice 1','Practice 2','Block 1','Block 2','Block 3','Block 4','Block 5']

# # Count the number of rows where 'Section' includes each of the substrings
# counts = {'# of trials in ' + substring: df['Section'].str.contains(substring).sum() for substring in substrings}

# # Count the number of rows where 'Section' does not include any of the substrings
# none_count = (~df['Section'].str.contains('|'.join(substrings))).sum()

# # Add the count of none to the dictionary
# counts['# of trials in others'] = none_count

# # Print out the counts
# print(counts)

# # Filter the DataFrame for rows where 'Section' includes 'Block 1'
# block1_df = df[df['Section'].str.contains('Block 1')]

# # Count the number of each different value in the 'Inhibition' column
# inhibition_counts = block1_df['Inhibition'].value_counts()
# print(inhibition_counts)




In [477]:
# Existing code to find duplicates
duplicates = df[df.duplicated(subset=['Block', 'TrialNumber'], keep=False)]
display(duplicates)

# Count the occurrences of each combination of 'Block' and 'TrialNumber'
dup_counts = duplicates.groupby(['Block', 'TrialNumber']).size()

# Iterate over each group of duplicates
for (block, trial), count in dup_counts.items():  # Use .items() instead of .iteritems()
    if count == 2:
        # If there are exactly 2 duplicates, remove the first occurrence
        first_index = df[(df['Block'] == block) & (df['TrialNumber'] == trial)].index[0]
        df = df.drop(first_index)
    elif count > 2:
        # If there are more than 2 duplicates, print an error message
        print(f"Error: More than 2 duplicates found for Block {block} and TrialNumber {trial}")

# Display the modified DataFrame
# display(df)

Section Start Time Section End Time  Duration (ms)  \
1455            49:41.2          49:41.9            687   
1456            49:41.9          49:42.3            375   
1494            50:09.8          50:10.5            734   
1495            50:10.5          50:10.8            329   
1517            50:26.3          50:27.0            736   
1518            50:27.0          50:27.3            331   

                                                  Event             Section  \
1455      Stimulus (Character:O) Displayed for 650.0 ms  Block 5 - Stimulus   
1456  Inhibit (Character:O) Displayed with Sound for...   Block 5 - Inhibit   
1494      Stimulus (Character:O) Displayed for 700.0 ms  Block 5 - Stimulus   
1495  Inhibit (Character:O) Displayed with Sound for...   Block 5 - Inhibit   
1517      Stimulus (Character:O) Displayed for 700.0 ms  Block 5 - Stimulus   
1518  Inhibit (Character:O) Displayed with Sound for...   Block 5 - Inhibit   

     User Response  Correct Key Correctness  StimulusRT   InhibitRT  ...  \
1455          Left  {LEFTARROW}   Incorrect         0.0  584.812045  ...   
1456          Left          NaN   Incorrect         0.0  584.812045  ...   
1494          Left  {LEFTARROW}   Incorrect         0.0  513.830900  ...   
1495          Left          NaN   Incorrect         0.0  513.830900  ...   
1517          Left  {LEFTARROW}   Incorrect         0.0  642.931104  ...   
1518          Left          NaN   Incorrect         0.0  642.931104  ...   

      PerRunIncorrect  InhibitACC  PerRunAcc StimDurSum  StimDurAvg  \
1455                1           1   0.900000       4250           0   
1456                2           0   0.818182       4900           0   
1494                2           1   0.846154       6150           0   
1495                3           0   0.785714       6850           0   
1517                3           1   0.800000       7500           0   
1518                4           0   0.750000       8200           0   

      PerRunStimDurAvg  DurationChange   RT_to_use  Post Error    Block  
1455        425.000000             NaN  584.812045        True  Block 5  
1456        445.454545        Decrease  584.812045        True  Block 5  
1494        473.076923             NaN  513.830900        True  Block 5  
1495        489.285714        Decrease  513.830900        True  Block 5  
1517        500.000000             NaN  642.931104        True  Block 5  
1518        512.500000        Decrease  642.931104        True  Block 5  

[6 rows x 34 columns]

In [478]:
# Existing code to find duplicates
duplicates = df[df.duplicated(subset=['Block', 'TrialNumber'], keep=False)]
display(duplicates)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # display(df)

Empty DataFrame
Columns: [Section Start Time, Section End Time, Duration (ms), Event, Section, User Response, Correct Key, Correctness, StimulusRT, InhibitRT, SDAN, App Version, App Build, Inhibition, StimulusDuration, InhibitDuration, AdjustmentAmount, RunningCorrect, RunningIncorrect, RunningAccuracy, PerRunGoRT, PerRunGoRTAve, TrialNumber, PerRunCorrect, PerRunIncorrect, InhibitACC, PerRunAcc, StimDurSum, StimDurAvg, PerRunStimDurAvg, DurationChange, RT_to_use, Post Error, Block]
Index: []

[0 rows x 34 columns]

In [479]:
# # Calculate average RT and standard deviation
# average_rt = df['RT_to_use'].mean()
# std_rt = df['RT_to_use'].std()

# # Filter out rows where RT is more than 3 std dev from the mean
# df_filtered = df[(df['RT_to_use'] >= (average_rt - 3 * std_rt)) & (df['RT_to_use'] <= (average_rt + 3 * std_rt))]




# # Count included and excluded rows
# count_included = df_filtered.shape[0]
# count_excluded = df.shape[0] - count_included
# df = df_filtered
# print(count_included)
# print(count_excluded)

In [480]:
import pandas as pd

# Calculate average RT and standard deviation
# average_rt = df['RT_to_use'].mean()
# std_rt = df['RT_to_use'].std()
# Calculate average and standard deviation only using non-zero values
average_rt = df.loc[df['RT_to_use'] != 0, 'RT_to_use'].mean()
std_rt = df.loc[df['RT_to_use'] != 0, 'RT_to_use'].std()

# Define boundaries for filtering
lower_bound = average_rt - 3 * std_rt
upper_bound = average_rt + 3 * std_rt

# Filter the DataFrame based on the new conditions
df_filtered = df[(df['RT_to_use'] == 0) | ((df['RT_to_use'] != 0) & (df['RT_to_use'] >= lower_bound) & (df['RT_to_use'] <= upper_bound))]

# Count included and excluded rows
count_included = df_filtered.shape[0]
count_excluded = df.shape[0] - count_included

# Replace the original dataframe with the filtered one
df = df_filtered

# Print the counts
print("Count Included:", count_included)
print("Count Excluded:", count_excluded)


Count Included: 439
Count Excluded: 1


In [481]:
# Adding a new column to df to show the number of standard deviations each row's 'RT_to_use' is from the mean
df['std_dev_from_mean'] = (df['RT_to_use'] - average_rt) / std_rt

In [482]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # display(df)

In [483]:
# List of substrings you are interested in
# substrings = ['Practice 1','Practice 2','Block 1','Block 2','Block 3','Block 4','Block 5']
substrings = ['Block 1','Block 2','Block 3','Block 4','Block 5']

# Count the number of rows where 'Section' includes each of the substrings
counts = {'# of trials in ' + substring: df['Section'].str.contains(substring).sum() for substring in substrings}

# Count the number of rows where 'Section' does not include any of the substrings
none_count = (~df['Section'].str.contains('|'.join(substrings))).sum()

# Add the count of none to the dictionary
counts['# of trials in others'] = none_count

# Print out the counts
print(counts)

# # Filter the DataFrame for rows where 'Section' includes 'Block 1'
# block1_df = df[df['Section'].str.contains('Block 1')]

# # Count the number of each different value in the 'Inhibition' column
# inhibition_counts = block1_df['Inhibition'].value_counts()
# print(inhibition_counts)

{'# of trials in Block 1': 88, '# of trials in Block 2': 88, '# of trials in Block 3': 88, '# of trials in Block 4': 88, '# of trials in Block 5': 87, '# of trials in others': 0}


In [484]:
for i,header in enumerate(headers):
    if header == "SDAN":
        result["SDAN"] = df["SDAN"].iloc[0]
        continue
    elif header == "Input File":
        result[header] = inFile
        continue
    elif header == 'Home vs InPerson':
        if 'athome' in inFile.lower():
            result[header] = "At Home"
        elif 'inperson' in inFile.lower():
            result[header] = "In Person"
        else:
            result[header] = ""
        continue    
    elif header == 'S1 vs S2':
        if 'S1' in inFile:
            result[header] = "S1"
        elif "S2" in inFile:
            result[header] = "S2"
        else:
            result[header] = ""
        continue    
    
    if 'Practice' in header:
        data = df[df['Section'].str.contains('Practice')]
    else:
        data = df[~df['Section'].str.contains('Practice')]

    if header == 'Go % Incorrect':
        # Creating a copy of the original DataFrame to avoid altering it
        df_copy = data.copy()
        df_copy = df_copy[df_copy['Inhibition'] == 'Normal']
        
        # Fill empty 'User Response' values in the copied DataFrame using iterrows
        # for i, row in df_copy.iterrows():
        #     if pd.isna(row['User Response']) or row['User Response'] == '':
        #         next_index = df_copy.index[df_copy.index.get_loc(i) + 1] if df_copy.index.get_loc(i) + 1 in df_copy.index else None
        #         if next_index is not None:
        #             df_copy.at[i, 'User Response'] = df_copy.at[next_index, 'User Response']
                    
        # Remove rows where 'User Response' is still empty or NaN
        a = sum(df_copy['Inhibition'] == 'Normal')
        print('# of Go trials:',a)
        df_copy.dropna(subset=['User Response'], inplace=True)
        print('# of Go trials (except trials with no response):',a - sum(df_copy['Inhibition'] == 'Normal'))
            
        # Adjust the condition to count 'Correct' and 'Incorrect' only when 'Inhibition' is 'Normal'
        df_copy['Result'] = 'Not Applicable'
        df_copy.loc[df_copy['Inhibition'] == 'Normal', 'Result'] = (
            ((df_copy['Correct Key'] == '{RIGHTARROW}') & (df_copy['User Response'] == 'Right')) |
            ((df_copy['Correct Key'] == '{LEFTARROW}') & (df_copy['User Response'] == 'Left'))
        ).replace({True: 'Correct', False: 'Incorrect'})

        # Debug
        for i, row in df_copy.iterrows():
            if row['Result'] != row['Correctness']:
                print(row)
                
        # Recalculate the number of correct and incorrect responses
        correct_count = df_copy[df_copy['Inhibition'] == 'Normal']['Result'].value_counts().get('Correct', 0)
        incorrect_count = df_copy[df_copy['Inhibition'] == 'Normal']['Result'].value_counts().get('Incorrect', 0)

        print('# of Go trials (Correct):',correct_count)
        print('# of Go trials (InCorrect except no response):',incorrect_count)
        
        # Recalculate the percentage of incorrect responses
        # result[header] = incorrect_count / (correct_count + incorrect_count) * 100 if (correct_count + incorrect_count) > 0 else 0
        result[header] = incorrect_count / a * 100 if a > 0 else 0
        continue
    elif header == 'Go % Omission':
        # Creating a copy of the original DataFrame to avoid altering it
        df_copy = data.copy()
        df_copy = df_copy[df_copy['Inhibition'] == 'Normal']
                            
        # Remove rows where 'User Response' is still empty or NaN
        a = sum(df_copy['Inhibition'] == 'Normal')
        print('# of Go trials:',a)
        df_copy.dropna(subset=['User Response'], inplace=True)
        b = a - sum(df_copy['Inhibition'] == 'Normal')
        result[header] = b / a * 100 if a > 0 else 0
    
    elif header == 'Stop % Incorrect':
        result[header] = 100 - result['Stop % Accuracy']
        continue    
    
    if 'Post Go Error' in header:
        # Use rows where "Post Error" is True and previous row's "Inhibition" is "Normal"
        post_error_data = data[data['Post Error'] & (data['Inhibition'].shift(1) == 'Normal') & (data['Inhibition'] == 'Normal')]
        display(post_error_data)
        if 'Go % Accuracy' in header:
            result[header] = (post_error_data['Correctness'] == 'Correct').sum() / len(post_error_data) * 100
        elif 'Go RT' in header:
            # Filter out rows where 'RT_to_use' is zero
            non_zero_data = post_error_data[post_error_data['RT_to_use'] != 0]
            
            # Calculate the mean of 'RT_to_use' for the non-zero values
            result[header] = non_zero_data['RT_to_use'].mean()
            
        elif 'Efficiency' in header:
            # Calculate Post Go Error Efficiency if data is not empty or NaN
            if pd.notna(result.get("Post Go Error Go % Accuracy")) and pd.notna(result.get("Post Go Error Go RT")):
                result[header] = result["Post Go Error Go % Accuracy"] / result["Post Go Error Go RT"]
            else:
                result[header] = " "
        continue
    elif 'Post Any Error' in header:
        # Use rows where "Post Error" is True
        post_any_error_data = data[data['Post Error'] & (data['Inhibition'] == 'Normal')]
        if '% Accuracy' in header:
            result[header] = (post_any_error_data['Correctness'] == 'Correct').sum() / len(post_any_error_data) * 100
        elif 'Go RT' in header:
            # Use only non-zero
            non_zero_post_any_error = post_any_error_data[post_any_error_data['RT_to_use'] != 0]
            # result[header] = post_any_error_data['RT_to_use'].mean()
            result[header] = non_zero_post_any_error['RT_to_use'].mean()
        elif 'Efficiency' in header:
            if pd.notna(result.get("Post Any Error Go % Accuracy")) and pd.notna(result.get("Post Any Error Go RT")):
                result[header] = result["Post Any Error Go % Accuracy"] / result["Post Any Error Go RT"] 
            else:
                result[header] = " "
        continue
        
    if 'Go' in header:
        # Use rows where "Inhibition" column is "Normal"
        data = data[data['Inhibition'] == 'Normal']
    elif 'Stop' in header:
        # Use rows where "Inhibition" column is "Inhibit"
        data = data[data['Inhibition'] == 'Inhibit']
    # else:
    #     # Use all rows
    #     data = data

    # if 'Average' in header and 'RT' in header:
    #     # Calculate average of 'RT_to_use' column
    #     result[header] = data['RT_to_use'].mean()
    if 'Average' in header and 'RT' in header:
        non_zero_data = data[data['RT_to_use'] != 0]
        result[header] = non_zero_data['RT_to_use'].mean()
    # elif 'Median' in header and 'RT' in header:
        # Calculate median of 'RT_to_use' column
        # Temporarily set options to display the entire DataFrame
        # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            # print(data['RT_to_use'])

        # # Count the number of zeros
        # num_zeros = (data['RT_to_use'] == 0).sum()
        # print("Number of zeros:", num_zeros)
        # print("Number of all:", (data['RT_to_use'] > -100000).sum())
        # print(data['RT_to_use'].median())
        
        
        # result[header] = data['RT_to_use'].median()
    elif 'Median' in header and 'RT' in header:
        non_zero_data = data[data['RT_to_use'] != 0]
        result[header] = non_zero_data['RT_to_use'].median()
    elif '% Accuracy' in header:
        # Calculate accuracy
        result[header] = (data['Correctness'] == 'Correct').sum() / len(data) * 100 if len(data) > 0 else np.nan
    elif header == 'Average SSD':
        # Calculate average of StimulusDuration column using only rows where "Inhibition" is "Inhibit"
        inhibit_data = data[data['Inhibition'] == 'Inhibit']
        result[header] = inhibit_data['StimulusDuration'].mean()
    elif header == 'SSRT':
        # Calculate SSRT
        result[header] = result.get("Median Go RT", np.nan) - result.get("Average SSD", np.nan)
    elif header == "Post Go Error Efficiency":
        # Calculate Post Go Error Efficiency if data is not empty or NaN
        if pd.notna(result.get("Post Go Error Go % Accuracy")) and pd.notna(result.get("Post Go Error Go RT")):
            result[header] = result["Post Go Error Go % Accuracy"] / result["Post Go Error Go RT"]
        else:
            result[header] = " "
    elif header == "Post Any Error Efficiency":
        if pd.notna(result.get("Post Any Error Go % Accuracy")) and pd.notna(result.get("Post Any Error Go RT")):
            result[header] = result["Post Any Error Go % Accuracy"] / result["Post Any Error Go RT"]
        else:
            result[header] = " "
    elif header == "Include/Exclude":
        # Check if all blocks are present (1,2,3,4,5)
        blocks_present = all(data['Section'].str.contains(f'Block {i}').any() for i in range(1, 6))

        # Calculate the percentage of 'Correct' in 'Normal' Inhibition rows
        normal_rows = data[data['Inhibition'] == 'Normal']
        normal_correct_percentage = (normal_rows['Correctness'] == 'Correct').sum() / len(normal_rows) * 100 if len(normal_rows) > 0 else 0
        result['normal_correct_percentage'] = normal_correct_percentage
        
        # Calculate the percentage of 'Correct' in 'Inhibit' Inhibition rows
        inhibit_rows = data[data['Inhibition'] == 'Inhibit']
        # inhibit_correct_percentage = (inhibit_rows['Correctness'] == 'Correct').sum() / len(inhibit_rows) * 100 if len(inhibit_rows) > 0 else 0
        # result['inhibit_correct_percentage'] = inhibit_correct_percentage
        inhibit_correct_number = (inhibit_rows['Correctness'] == 'Correct').sum() if len(inhibit_rows) > 0 else 0

        # Decide to 'Include' or 'Exclude' based on conditions
        if blocks_present and normal_correct_percentage >= 60 and inhibit_correct_number >= 30:
            result[header] = "Include"
        else:
            result[header] = "Exclude"
    elif "Post Go Error" in header:
        # Use rows where "Post Error" is True and previous row's "Inhibition" is "Normal"
        post_error_data = data[data['Post Error'] & (data['Inhibition'].shift(1) == 'Normal')]
        if 'Go % Accuracy' in header:
            result[header] = post_error_data['Correctness'] == 'Correct'.sum() / len(post_error_data) * 100
        # elif 'Go RT' in header:
        #     result[header] = post_error_data['RT_to_use'].mean()
        elif 'Go RT' in header:
            non_zero_post_error = post_error_data[post_error_data['RT_to_use'] != 0]
            result[header] = non_zero_post_error['RT_to_use'].mean()
    
    elif "Post Any Error" in header:
        # Use rows where "Post Error" is True
        post_any_error_data = data[data['Post Error']]
        if '% Accuracy' in header:
            result[header] = post_any_error_data['Correctness'] == 'Correct'.sum() / len(post_any_error_data) * 100
        # elif 'Go RT' in header:
        #     result[header] = post_any_error_data['RT_to_use'].mean()
        elif 'Go RT' in header:
            non_zero_post_any_error = post_any_error_data[post_any_error_data['RT_to_use'] != 0]
            result[header] = non_zero_post_any_error['RT_to_use'].mean()

    elif "Error Go RT" in header:
        # Calculate average of 'RT_to_use' where there is an error
        error_data = data[data['Correctness'] == 'Incorrect']
        # result[header] = error_data['RT_to_use'].mean()
        non_zero_error_data = error_data[error_data['RT_to_use'] != 0]
        result[header] = non_zero_error_data['RT_to_use'].mean()

# of Go trials: 330
# of Go trials (except trials with no response): 242
# of Go trials (Correct): 88
# of Go trials (InCorrect except no response): 0
# of Go trials: 330


Section Start Time Section End Time  Duration (ms)  \
174             33:23.1          33:24.2           1038   
187             33:32.5          33:33.6           1035   
190             33:34.8          33:35.8           1033   
193             33:37.2          33:38.2           1039   
214             33:51.3          33:52.3           1038   
...                 ...              ...            ...   
1553            50:52.2          50:53.2           1032   
1556            50:54.4          50:55.4           1033   
1570            51:04.1          51:05.1           1024   
1584            51:13.4          51:14.4           1037   
1587            51:15.8          51:16.8           1033   

                                               Event             Section  \
174   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
187   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
190   Stimulus (Character:O) Displayed for 1000.0 ms  Block 1 - Stimulus   
193   Stimulus (Character:O) Displayed for 1000.0 ms  Block 1 - Stimulus   
214   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
...                                              ...                 ...   
1553  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1556  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1570  Stimulus (Character:O) Displayed for 1000.0 ms  Block 5 - Stimulus   
1584  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1587  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   

     User Response   Correct Key Correctness  StimulusRT  InhibitRT  ...  \
174          Right  {RIGHTARROW}     Correct  747.337937        0.0  ...   
187            NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
190            NaN   {LEFTARROW}   Incorrect    0.000000        0.0  ...   
193           Left   {LEFTARROW}     Correct  599.186897        0.0  ...   
214            NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
...            ...           ...         ...         ...        ...  ...   
1553           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1556           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1570           NaN   {LEFTARROW}   Incorrect    0.000000        0.0  ...   
1584           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1587           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   

      InhibitACC  PerRunAcc  StimDurSum StimDurAvg  PerRunStimDurAvg  \
174            0   0.000000           0          0        250.000000   
187            1   1.000000         250          0        250.000000   
190            1   1.000000         250          0        250.000000   
193            1   1.000000         250          0        250.000000   
214            1   0.750000        1100          0        275.000000   
...          ...        ...         ...        ...               ...   
1553           0   0.722222        9550          0        530.555556   
1556           0   0.722222        9550          0        530.555556   
1570           1   0.750000       10900          0        545.000000   
1584           1   0.772727       12450          0        565.909091   
1587           1   0.772727       12450          0        565.909091   

      DurationChange   RT_to_use  Post Error    Block  std_dev_from_mean  
174              NaN  747.337937        True  Block 1           0.226638  
187              NaN    0.000000        True  Block 1          -4.061310  
190              NaN    0.000000        True  Block 1          -4.061310  
193              NaN  599.186897        True  Block 1          -0.623398  
214              NaN    0.000000        True  Block 1          -4.061310  
...              ...         ...         ...      ...                ...  
1553             NaN    0.000000        True  Block 5          -4.061310  
1556 

Section Start Time Section End Time  Duration (ms)  \
174             33:23.1          33:24.2           1038   
187             33:32.5          33:33.6           1035   
190             33:34.8          33:35.8           1033   
193             33:37.2          33:38.2           1039   
214             33:51.3          33:52.3           1038   
...                 ...              ...            ...   
1553            50:52.2          50:53.2           1032   
1556            50:54.4          50:55.4           1033   
1570            51:04.1          51:05.1           1024   
1584            51:13.4          51:14.4           1037   
1587            51:15.8          51:16.8           1033   

                                               Event             Section  \
174   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
187   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
190   Stimulus (Character:O) Displayed for 1000.0 ms  Block 1 - Stimulus   
193   Stimulus (Character:O) Displayed for 1000.0 ms  Block 1 - Stimulus   
214   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
...                                              ...                 ...   
1553  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1556  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1570  Stimulus (Character:O) Displayed for 1000.0 ms  Block 5 - Stimulus   
1584  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1587  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   

     User Response   Correct Key Correctness  StimulusRT  InhibitRT  ...  \
174          Right  {RIGHTARROW}     Correct  747.337937        0.0  ...   
187            NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
190            NaN   {LEFTARROW}   Incorrect    0.000000        0.0  ...   
193           Left   {LEFTARROW}     Correct  599.186897        0.0  ...   
214            NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
...            ...           ...         ...         ...        ...  ...   
1553           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1556           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1570           NaN   {LEFTARROW}   Incorrect    0.000000        0.0  ...   
1584           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1587           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   

      InhibitACC  PerRunAcc  StimDurSum StimDurAvg  PerRunStimDurAvg  \
174            0   0.000000           0          0        250.000000   
187            1   1.000000         250          0        250.000000   
190            1   1.000000         250          0        250.000000   
193            1   1.000000         250          0        250.000000   
214            1   0.750000        1100          0        275.000000   
...          ...        ...         ...        ...               ...   
1553           0   0.722222        9550          0        530.555556   
1556           0   0.722222        9550          0        530.555556   
1570           1   0.750000       10900          0        545.000000   
1584           1   0.772727       12450          0        565.909091   
1587           1   0.772727       12450          0        565.909091   

      DurationChange   RT_to_use  Post Error    Block  std_dev_from_mean  
174              NaN  747.337937        True  Block 1           0.226638  
187              NaN    0.000000        True  Block 1          -4.061310  
190              NaN    0.000000        True  Block 1          -4.061310  
193              NaN  599.186897        True  Block 1          -0.623398  
214              NaN    0.000000        True  Block 1          -4.061310  
...              ...         ...         ...      ...                ...  
1553             NaN    0.000000        True  Block 5          -4.061310  
1556 

Section Start Time Section End Time  Duration (ms)  \
174             33:23.1          33:24.2           1038   
187             33:32.5          33:33.6           1035   
190             33:34.8          33:35.8           1033   
193             33:37.2          33:38.2           1039   
214             33:51.3          33:52.3           1038   
...                 ...              ...            ...   
1553            50:52.2          50:53.2           1032   
1556            50:54.4          50:55.4           1033   
1570            51:04.1          51:05.1           1024   
1584            51:13.4          51:14.4           1037   
1587            51:15.8          51:16.8           1033   

                                               Event             Section  \
174   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
187   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
190   Stimulus (Character:O) Displayed for 1000.0 ms  Block 1 - Stimulus   
193   Stimulus (Character:O) Displayed for 1000.0 ms  Block 1 - Stimulus   
214   Stimulus (Character:X) Displayed for 1000.0 ms  Block 1 - Stimulus   
...                                              ...                 ...   
1553  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1556  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1570  Stimulus (Character:O) Displayed for 1000.0 ms  Block 5 - Stimulus   
1584  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   
1587  Stimulus (Character:X) Displayed for 1000.0 ms  Block 5 - Stimulus   

     User Response   Correct Key Correctness  StimulusRT  InhibitRT  ...  \
174          Right  {RIGHTARROW}     Correct  747.337937        0.0  ...   
187            NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
190            NaN   {LEFTARROW}   Incorrect    0.000000        0.0  ...   
193           Left   {LEFTARROW}     Correct  599.186897        0.0  ...   
214            NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
...            ...           ...         ...         ...        ...  ...   
1553           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1556           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1570           NaN   {LEFTARROW}   Incorrect    0.000000        0.0  ...   
1584           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   
1587           NaN  {RIGHTARROW}   Incorrect    0.000000        0.0  ...   

      InhibitACC  PerRunAcc  StimDurSum StimDurAvg  PerRunStimDurAvg  \
174            0   0.000000           0          0        250.000000   
187            1   1.000000         250          0        250.000000   
190            1   1.000000         250          0        250.000000   
193            1   1.000000         250          0        250.000000   
214            1   0.750000        1100          0        275.000000   
...          ...        ...         ...        ...               ...   
1553           0   0.722222        9550          0        530.555556   
1556           0   0.722222        9550          0        530.555556   
1570           1   0.750000       10900          0        545.000000   
1584           1   0.772727       12450          0        565.909091   
1587           1   0.772727       12450          0        565.909091   

      DurationChange   RT_to_use  Post Error    Block  std_dev_from_mean  
174              NaN  747.337937        True  Block 1           0.226638  
187              NaN    0.000000        True  Block 1          -4.061310  
190              NaN    0.000000        True  Block 1          -4.061310  
193              NaN  599.186897        True  Block 1          -0.623398  
214              NaN    0.000000        True  Block 1          -4.061310  
...              ...         ...         ...      ...                ...  
1553             NaN    0.000000        True  Block 5          -4.061310  
1556 

In [485]:
# Concatenate result and dict_VAS.
result.update(dict_VAS)
result['count_included'] = count_included
result['count_excluded'] = count_excluded
result.update(counts)

In [486]:
display(result)
print(result['Go % Accuracy'] + result['Go % Incorrect'] + result['Go % Omission'])

{'SDAN': 24626,
 'Input File': 'input/24626_9.6.23_InPerson_S1.csv',
 'Home vs InPerson': 'In Person',
 'S1 vs S2': 'S1',
 'Average Run Go RT': 640.3208564647728,
 'Stop % Accuracy': 73.39449541284404,
 'Stop % Incorrect': 26.605504587155963,
 'Go % Accuracy': 26.666666666666668,
 'Go % Incorrect': 0.0,
 'Go % Omission': 73.33333333333333,
 'Median Go RT': 622.76202445,
 'Average SSD': 412.38532110091745,
 'SSRT': 210.37670334908256,
 'Post Go Error Go % Accuracy': 20.105820105820104,
 'Post Go Error Go RT': 670.3870076842105,
 'Post Go Error Efficiency': 0.029991363011753147,
 'Post Any Error Go % Accuracy': 21.35922330097087,
 'Post Any Error Go RT': 669.6270758409091,
 'Post Any Error Efficiency': 0.031897191842412034,
 'normal_correct_percentage': 26.666666666666668,
 'Include/Exclude': 'Exclude',
 'Pre_Tense': '13',
 'Pre_Angry': '0',
 'Pre_Annoyed': '2',
 'Pre_Happy': '67',
 'Pre_Tired': '16',
 'Mid_Tense': '54',
 'Mid_Angry': '30',
 'Mid_Annoyed': '50',
 'Mid_Happy': '51',
 'Mid

100.0


In [487]:
from datetime import datetime
import os

series = pd.Series(result)
df_result = series.to_frame().transpose()

# Get the current date and time
now = datetime.now()

# Format the current date and time as a string, e.g., "20230926_142530"
datetime_str = now.strftime('%Y%m%d_%H%M%S')

# Construct the filename using the date and time string<?xml version="1.0" encoding="UTF-8"?>
filename = f'sst_mobile_summary_{datetime_str}.csv'

# Save the DataFrame to a CSV file with the constructed filename
df_result.to_csv(filename, index=False)

# Assume the directory you want to save in is the current working directory
directory = os.getcwd()

# Create the absolute path
absolute_file_path = os.path.abspath(os.path.join(directory, filename))

# Now absolute_file_path contains the absolute path to the file
print(absolute_file_path)

/Users/leek13/github/BehaviorProcessing/StopSignalTask/mobile/sst_mobile_summary_20240105_095559.csv
